In [1]:
import polars as pl
import altair as alt

In [3]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [5]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).join(
    pl.scan_parquet(
        "data/projevy_jazyky.parquet",
    ),
    how="left",
    on="text"
).filter(
    pl.col("jazyk") != "sk"
).collect()

## Jak se mluví o důvěře a nedůvěře vládě

In [39]:
kombo = pl.concat(
    [
        df.filter(
            pl.col("text").str.contains("(?i)[^e]důvě\w{0,10} vlád")
        ).group_by(
            pl.col("datum").dt.year()
        ).len(
        ).with_columns(
            pl.lit("důvěra").alias("duvera")
        ),
        df.filter(
            pl.col("text").str.contains("(?i)edůvě\w{0,10} vlád")
        ).group_by(
            pl.col("datum").dt.year()
        ).len(
        ).with_columns(
            pl.lit("nedůvěra").alias("duvera")
        )
    ]
).sort(by='datum')

<>:4: SyntaxWarning: invalid escape sequence '\w'
<>:12: SyntaxWarning: invalid escape sequence '\w'
<>:4: SyntaxWarning: invalid escape sequence '\w'
<>:12: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37664\3420974115.py:4: SyntaxWarning: invalid escape sequence '\w'
  pl.col("text").str.contains("(?i)[^e]důvě\w{0,10} vlád")
C:\Users\micha\AppData\Local\Temp\ipykernel_37664\3420974115.py:12: SyntaxWarning: invalid escape sequence '\w'
  pl.col("text").str.contains("(?i)edůvě\w{0,10} vlád")


In [41]:
alt.Chart(
    kombo,
    width=800
).mark_bar(
).encode(
    alt.X("datum:Q"),
    alt.Y("len:Q"),
    alt.Color("duvera:N")
)

alt.Chart(...)

## Hlasování o (ne)důvěře

In [44]:
alt.Chart(
    df.filter(
            pl.col("text").str.contains("(?i)hlasov\w{0,10} o (ne|)důvě\w{0,10} vlád")
        ).group_by(
            pl.col("datum").dt.year()
        ).len(
        ),
    width=800
).mark_bar(
).encode(
    alt.X("datum:Q"),
    alt.Y("len:Q"),
    alt.Color("duvera:N")
)

<>:3: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_37664\1674948333.py:3: SyntaxWarning: invalid escape sequence '\w'
  pl.col("text").str.contains("(?i)hlasov\w{0,10} o (ne|)důvě\w{0,10} vlád")


alt.Chart(...)